In [ ]:
import pandas as pd
import random
# 固定随机数种子
random.seed(24)
user_count = 1000
a_preferences = ([round(random.randint(1, 10), 1) for _ in range(user_count)]) 
b_preferences = ([round(random.randint(1, 10), 1) for _ in range(user_count)]) 
c_preferences = ([round(random.randint(1, 10), 1) for _ in range(user_count)])

# 已售出的 A、B、C 产品数量
sold_a = 0
sold_b = 0
sold_c = 0
# 已售出的 A、B、C 产生的价值
a_profits = 0
b_profits = 0
c_profits = 0

# 处理前 10 名顾客
for i in range(10):
    if a_preferences[i] >= b_preferences[i] and a_preferences[i] >= c_preferences[i]:
        sold_a += 1
        a_profits += a_preferences[i] 
    elif b_preferences[i] >= a_preferences[i] and b_preferences[i] >= c_preferences[i]:
        sold_b += 1
        b_profits += b_preferences[i] 
    elif c_preferences[i] >= a_preferences[i] and c_preferences[i] >= b_preferences[i]:
        sold_c += 1
        c_profits += c_preferences[i] 

# 处理剩余顾客 每次10名
for r in range(999):
    # 计算每轮需要保持的 A、B、C 产品数量
    target_a = 5 * (r + 2)
    target_b = 3 * (r + 2)
    target_c = 2 * (r + 2)

    # 处理每轮的 10 名顾客
    for i in range(10 * r + 10, 10 * (r + 1) + 10):
        #假如A,B均已经超出 分给C
        if sold_a >= target_a and sold_b >= target_b:
            sold_c += 1
            c_profits += c_preferences[i]
        #假如A,C均已经超出 分给B
        elif sold_a >= target_a and sold_c >= target_c:
            sold_b += 1
            b_profits += b_preferences[i] 
        #假如B,C均已经超出 分给A
        elif sold_b >= target_b and sold_c >= target_c:
            sold_a += 1
            a_profits += a_preferences[i] 
        
        #假如只有A已经超出 比较B，C后分配   
        elif sold_a >= target_a:
            if b_preferences[i] >= c_preferences[i]:
                sold_b += 1
                b_profits += b_preferences[i] 
            else:
                sold_c += 1
                c_profits += c_preferences[i]
        
        #假如只有B已经超出 比较A，C后分配   
        elif sold_b >= target_b:
            if a_preferences[i] >= c_preferences[i]:
                sold_a += 1
                a_profits += a_preferences[i] 
            else:
                sold_c += 1
                c_profits += c_preferences[i]
        #假如只有C已经超出 比较A，B后分配   
        elif sold_c >= target_c:
            if a_preferences[i] >= b_preferences[i]:
                sold_a += 1
                a_profits += a_preferences[i] 
            else:
                sold_b += 1
                b_profits += b_preferences[i]
                
        #假设均没有超出
        elif sold_a <= target_a and a_preferences[i] >= b_preferences[i] and a_preferences[i] >= c_preferences[i]:
            sold_a += 1
            a_profits += a_preferences[i] 
        elif sold_b <= target_b and b_preferences[i] >= a_preferences[i] and b_preferences[i] >= c_preferences[i]:
            sold_b += 1
            b_profits += b_preferences[i] 
        elif sold_c <= target_c and c_preferences[i] >= a_preferences[i] and c_preferences[i] >= b_preferences[i]:
            sold_c += 1
            c_profits += c_preferences[i]     
        else:
            print('匹配不成功', sold_a, target_a, sold_b, target_b, sold_c, target_c)
            print(a_preferences[i], b_preferences[i], c_preferences[i])
            
total_profit = a_profits + b_profits + c_profits         
print("总利润:", total_profit)

In [ ]:
customer_count = 3000

A_value = pd.DataFrame([round(random.randint(1, 10), 1) for _ in range(customer_count)]) * 5 / 10
B_value = pd.DataFrame([round(random.randint(1, 10), 1) for _ in range(customer_count)]) * 3 / 10
C_value = pd.DataFrame([round(random.randint(1, 10), 1) for _ in range(customer_count)]) * 1 / 10
D_value = pd.DataFrame([round(random.randint(1, 10), 1) for _ in range(customer_count)]) * 1 / 10
customers = [str(i) for i in range(customer_count)]
#各产品数量
A_count = customer_count / 2
B_count = customer_count / 2
C_count = customer_count / 2
D_count = customer_count / 2

import pulp

# 定义客户列表和产品库存
# customers = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']
profits_A = A_value[0].values
profits_B = B_value[0].values
profits_C = C_value[0].values
profits_D = D_value[0].values
# 创造线性规划问题实例
model = pulp.LpProblem("Maximize_Profits", pulp.LpMaximize)

# 决策变量：每种产品每位顾客是否购买
x_A = pulp.LpVariable.dicts("x_A", customers, cat=pulp.LpBinary)
x_B = pulp.LpVariable.dicts("x_B", customers, cat=pulp.LpBinary)
x_C = pulp.LpVariable.dicts("x_C", customers, cat=pulp.LpBinary)
x_D = pulp.LpVariable.dicts("x_D", customers, cat=pulp.LpBinary)

# 目标函数
model += pulp.lpSum([profits_A[i] * x_A[c] + profits_B[i] * x_B[c] + profits_C[i] * x_C[c] + profits_D[i] * x_D[c] for i, c in enumerate(customers)])

# 约束条件
# 每位顾客需要买2种产品
for c in customers:
    model += ((x_A[c] + x_B[c] + x_C[c] + x_D[c]) == 2, f"One_product_per_customer_{c}")

# 每种产品的销售数量限制
model += (pulp.lpSum([x_A[c] for c in customers]) == A_count , "Stock_limit_A")
model += (pulp.lpSum([x_B[c] for c in customers]) == B_count, "Stock_limit_B")
model += (pulp.lpSum([x_C[c] for c in customers]) == C_count, "Stock_limit_C")
model += (pulp.lpSum([x_D[c] for c in customers]) == D_count, "Stock_limit_D")
# 求解
model.solve()

# 输出解决方案和总利润
assignments = {}
for p, x in zip(["A", "B", "C", "D"], [x_A, x_B, x_C, x_D]):
#     print(f"Product {p}:")
    for c in customers:
        if x[c].value() == 1:
            if c in assignments:
                assignments[c].append(p)
            else:
                assignments[c] = [p]
# 输出每个顾客的购买组合及计算总利润
assignments, pulp.value(model.objective)
